# <font class="markdown-google-sans" size=7>**gif-captions** 🔠🖼️</font>

> *Pack of scripts providing GIF captions generation.*

<br><hr><br>

<a href="https://github.com/kubinka0505/gif-captions"><img src="https://img.shields.io/badge/github-open-lightgray?logo=github&logoColor=white&style=for-the-badge"></a>

# <div class="markdown-google-sans"><font color=lime>**1.**</font> Re/install requirements 📥</div>

In [ ]:
#@title <div class="markdown-google-sans">Setup code ⚙️</div>
import os
import warnings
from time import time
from pathlib import Path
from shutil import rmtree
from IPython import display
from zipfile import ZipFile
from requests import Session
from rich import print as rprint

try:
	from google.colab import files
	is_colab = True
except ImportError:
	is_colab = False

warnings.filterwarnings("ignore")

#-=-=-=-#
# Metadata

__title__	= "gif-captions"
__author__	= "kubinka0505"
__credits__ = __author__
__version__ = "1.0"
__date__	= "02nd March 2024"

#-=-=-=-#
# Code

InternetSession = Session()
InternetSession.headers.update(
	{
		"User-Agent":
		"Mozilla/5.0 (Windows NT 10.0; Win32; x32) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
	}
)

# Directories

directory_main = os.getcwd()
directory_code = os.path.join(directory_main, __title__)
directory_inputs = os.path.join(directory_main, "_inputs_uploaded")
directory_outputs = os.path.join(directory_main, "_outputs")

for directory in [directory_code, directory_inputs, directory_outputs]:
	os.makedirs(directory, exist_ok = True)

os.environ["DIRECTORY_MAIN"] = directory_main
os.environ["DIRECTORY_CODE"] = directory_code
os.environ["DIRECTORY_OUTPUTS"] = directory_outputs

#-=-=-=-#
# Functions

def is_URL(url: str) -> bool:
	"""
	Verifies whether `url` is URL.

	Args:
		url: String URL address.

	Returns:
		bool: True if `url` is URL, otherwise False.
	"""
	for delimiter in ["https://", "http://"]:
		url = url.split(delimiter)[-1]
	url = "https://" + url

	try:
		retval = InternetSession.head(url, allow_redirects = True).ok
	except Exception as error:
		retval = False

	return retval

In [ ]:
#@title <div class="markdown-google-sans">Repository 🗃️</div>

#-=-=-=-#
# Clone

if os.path.exists(directory_code):
	rmtree(directory_code)

!git clone https://github.com/kubinka0505/gif-captions $DIRECTORY_CODE
os.chdir(directory_code)

print()

!python3 -m pip install rich -r requirements.txt --progress-bar off | grep -v "already"
!apt-get install --reinstall ffmpeg sox pngquant gifsicle -qq > /dev/null 2 >&1

os.chdir(directory_main)

#-=-=-=-#
# Variables

main_file = [
	str(file.resolve())
	for file in Path(directory_code).glob("*.py")
][0]

os.environ["MAIN_FILE"] = main_file

# <div class="markdown-google-sans"><font color=gold>**2.**</font> Upload files 📤</font></div>

In [ ]:
#@markdown > ℹ️ Leave empty for manual upload.

URL_Visual = "https://kubinka0505.github.io/assets/media/img/png/Banners/Banner.png" #@param {type: "string"}
URL_Audio_Optional = "" #@param {type: "string"}

#@markdown ---

Ignore_audio_upload_if_field_is_empty = False #@param {type: "boolean"}

#-=-=-=-#

os.chdir(directory_inputs)

# Visual
upload_visual = 0

if not is_URL(URL_Visual):
	rprint("[b #FF0000]Visual file URL is empty or invalid![/]")
	print()
	rprint("Upload [b #44AAFF]VISUAL[/] file")

	upload_visual = 1
	URL_Visual = list(files.upload().keys())
	print()
	if len(URL_Visual) < 1:
		rprint("[b #FF0000]Visual input file was not provided![/]")
		raise SystemExit()
	else:
		URL_Visual = URL_Visual[0]
		URL_Visual = os.path.join(directory_inputs, URL_Visual)

# Audio (optional)
URL_Audio = ""

if not is_URL(URL_Audio_Optional):
	if not Ignore_audio_upload_if_field_is_empty:
		if upload_visual:
			print("\n" + "─" * 32 + "\n")
		rprint("[b #FFCC00]Audio file URL is empty or invalid![/]")
		print()
		rprint("Upload [b #44AAFF]AUDIO[/] file")

		URL_Audio = list(files.upload().keys())
		print()
		if len(URL_Audio) < 1:
			rprint("[b #FFCC00]Audio input file was not provided![/]")
			URL_Audio = ""
		else:
			URL_Audio = URL_Audio[0]

if URL_Audio:
	URL_Audio = os.path.join(directory_inputs, URL_Audio)

#-=-=-=-#

print(URL_Visual)
print(URL_Audio)

os.chdir(directory_main)

# <div class="markdown-google-sans"><font color=gold>**3.**</font> Generate ➕</div>

In [ ]:
#@title <div class="markdown-google-sans">Configure 📝</div>

#@markdown Required
Text = "An example text :snake:" #@param {type: "string"}

#@markdown Optional
Wrap = 18 #@param {type: "slider", min: 10, max: 25, step: 1}

#@markdown Optimization
Audio_Bitrate = 224 #@param {type: "slider", min: 96, max: 320, step: 16}
Delay_Count = 0 #@param {type: "slider", min: 2, max: 100, step: 1}
Loop_Count = 0 #@param {type: "slider", min: 0, max: 50, step: 1}
Lossy = 200 #@param {type: "slider", min: 1, max: 200, step: 1}

#@markdown Styling
Font = "Futura" #@param ["Futura", "Roboto"]
Text_Color = "#000" #@param {type: "string"}
Background_Color = "#FFF" #@param {type: "string"}
Emoji_Style = "Twitter" #@param ["Apple", "Google", "Microsoft", "Samsung", "WhatsApp", "Twitter", "Facebook", "JoyPixels", "OpenMoji", "EmojiDex", "LG", "HTC", "Mozilla"]
Width = 0 #@param {type: "slider", min: 0, max: 2500, step: 1}

#@markdown Switches
Force_GIF = False #@param {type: "boolean"}
Encode_WebM = False #@param {type: "boolean"}
Keep_Session_Directory = False #@param {type: "boolean"}
Optimization = True #@param {type: "boolean"}

#-=-=-=-#
# Handling

if not Text:
	rprint("[b #FF0000]Text was not provided![/]")
	raise SystemExit()

#-=-=-=-#

# Required
os.environ["INPUT_VISUAL"] = URL_Visual
os.environ["TEXT"] = Text

# Optional
os.environ["INPUT_AUDIO"] = URL_Audio
os.environ["TEXT_WRAP"] = str(Wrap)

# Optimization
os.environ["AUDIO_BITRATE"] = str(Audio_Bitrate)
os.environ["DELAY_COUNT"] = str(Delay_Count)
os.environ["LOOP_COUNT"] = str(Loop_Count)
os.environ["LOSSY"] = str(Lossy)

# Styling
os.environ["FONT"] = Font + ".otf"
os.environ["TEXT_COLOR"] = Text_Color
os.environ["BACKGROUND_COLOR"] = Background_Color
os.environ["EMOJI_STYLE"] = Emoji_Style
os.environ["WIDTH"] = str(Width)

# Switches
os.environ["FORCE_GIF"] = "-togif" if Force_GIF else ""
os.environ["ENCODE_WEBM"] = "-webm" if Encode_WebM else ""
os.environ["KEEP_SESSION_DIRECTORY"] = "-kc" if Keep_Session_Directory else ""
os.environ["OPTIMIZATION"] = "" if Optimization else "-nox"

In [ ]:
#@title <div class="markdown-google-sans">Run ▶️</div>

Verbosity = "1 (Debug)" #@param ["-1 (Silent)", "0 (Info)", "1 (Debug)"]
Show_Help = False #@param {type: "boolean"}

Verbosity = Verbosity.split(" ")[0]

#-=-=-=-#

os.environ["COMMAND_PYTHON"] = os.sys.executable
os.environ["COMMAND"] = os.sys.executable
if Show_Help:
	os.environ["COMMAND"] = "echo"

!$COMMAND $MAIN_FILE \
	-i "$INPUT_VISUAL" -a "$INPUT_AUDIO" \
	-t "$TEXT" -w $TEXT_WRAP -sw $WIDTH \
	-ab $AUDIO_BITRATE \
	-r $DELAY_COUNT -l $LOOP_COUNT -lsy $LOSSY \
	-f "$FONT" -fg-col "$TEXT_COLOR" -bg-col "$BACKGROUND_COLOR" -s "$EMOJI_STYLE" \
	$FORCE_GIF $ENCODE_WEBM $KEEP_SESSION_DIRECTORY $OPTIMIZATION $HELP \
	-od "$DIRECTORY_OUTPUTS" -v $VERBOSITY -nod -fpl

print()

if Show_Help:
	!$COMMAND_PYTHON $MAIN_FILE -h

# <div class="markdown-google-sans"><font color=lime>**4.**</font> Display latest output file 🖥️</div>


In [ ]:
#@markdown > #### ⚠️ <font color=red>**Due to the base64 encoded output string, running this cell may lead to webpage responsibility failure.**</font>
#@markdown > Clear this cell output to revert.
#@markdown ---

Autoplay_video = False #@param {type: "boolean"}
Autoplay_video = "autoplay" if Autoplay_video else ""

#-=-=-=-#

output_file = sorted(
	[
	str(File.resolve())
	for File in Path(directory_outputs).rglob("*.*")
		if File.is_file()
	],

	key = os.path.getmtime
)[-1]

output_format = os.path.splitext(output_file)[1].strip(".")

#-=-=-=-#
# Show

display_size = 320
is_video = output_format.lower().endswith(("webm", "mp4"))

# Does not work with normal syntax, wontfix
display.Video(
	output_file,
	width = display_size,
	embed = True,
	html_attributes = f"controls {Autoplay_video}".strip()
) if is_video else \
display.Image(
	filename = output_file,
	width = display_size,
	embed = True
)

---

# <div class="markdown-google-sans">Utils ⚙️</div>

In [ ]:
#@title <div class="markdown-google-sans">Download archive with output files 🗃️</div>
Emphasize_archive_file_name = True #@param {type: "boolean"}
beggining = "!_" if Emphasize_archive_file_name else ""

#@markdown > ℹ️ Pushes the archive file to top of the explorer's file list (**if** it's sorted by name) by putting `!_` to its file name beggining.

#-=-=-=-#

__padding = 7
archive_path = beggining + "_".join((__title__, str(time()).split(".")[1][:__padding].rjust(__padding, "0")))
archive_path += ".zip"
archive_path = os.path.abspath(archive_path)

#-=-=-=-#

# Archive Creation
print("Making Archive...")
with ZipFile(archive_path, "w", 14, 1, 9) as archive:
	for file in Path(directory_outputs).rglob("*.*"):
		file = str(file.resolve())

		if os.path.isfile(file):
			ArcName = os.path.relpath(file, directory_outputs)

			rprint(f'\t[b i]Writing [#44AACC]"{ArcName}"[/]...')
			archive.write(file, ArcName)

#-=-=-=-#

files.download(archive_path)